### Imports

In [1]:
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 180 µs (started: 2023-06-12 22:06:23 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = "greedy"
print("Method:\t\t", method)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = ""
print("Option:\t", option)

Exp:		 exp5
Method:		 greedy
Groups id:	 ['s_WT', 's_zwf1^']
Subgroups id:	 {'s_WT': ['1', '2', '3', '4', '5'], 's_zwf1^': ['1', '2', '3']}
Option:	 
time: 2.77 ms (started: 2023-06-12 22:06:23 -05:00)


### Get common subgraphs

In [3]:
# read edges
dict_graphs = {}

for group in tqdm(groups_id):
    graphs = []
    for subgroup in tqdm(subgroups_id[group]):
        df_edges = pd.read_csv("output/{}/preprocessing/edges/edges_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"source": "string", "target": "string"})
        G = nx.from_pandas_edgelist(df_edges, "source", "target", edge_attr="weight")
        # graph_detail(G)
        graphs.append(G)
    
    dict_graphs[group] = graphs

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:03<00:00,  1.56s/it]

time: 3.14 s (started: 2023-06-12 22:06:23 -05:00)


In [4]:
# get subgraphs

dict_df_edges_filter = get_subgraphs_global(dict_graphs, groups_id)

 50%|█████     | 1/2 [00:01<00:01,  1.88s/it]

Num. nodes: 902
Num. edges: 1247



100%|██████████| 2/2 [00:03<00:00,  1.52s/it]

Num. nodes: 1000
Num. edges: 13664

time: 3.05 s (started: 2023-06-12 22:06:26 -05:00)


In [5]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,411,485
1,411,359
2,411,377
3,411,483
4,411,712


time: 5.78 ms (started: 2023-06-12 22:06:29 -05:00)


In [6]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 2/2 [00:00<00:00, 209.99it/s]

time: 24.6 ms (started: 2023-06-12 22:06:29 -05:00)


In [7]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 2/2 [00:02<00:00,  1.19s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
479,0,208,0.938184,0.761635,0.612201,0.521909,0.703441
804,0,57,0.670125,0.705567,0.710467,0.874121,0.863436
944,0,721,0.592912,0.907517,0.692873,0.959390,0.884151
943,0,946,0.601799,0.698748,0.922260,0.671509,0.935914
1206,1,557,0.818831,0.542379,0.864796,0.881613,0.539361


time: 2.39 s (started: 2023-06-12 22:06:29 -05:00)


### Filter by STD and average weight

In [8]:
# dispersion (std)

dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 2/2 [00:00<00:00,  3.84it/s]


,source,target,weight
0,0,208,0.707474
1,0,57,0.764744
2,0,721,0.807369
3,0,946,0.766046
4,1,557,0.729396


<Figure size 640x480 with 0 Axes>

time: 529 ms (started: 2023-06-12 22:06:32 -05:00)


---

### STD-ANOVA

STD

In [9]:
# STD
    
""" df_edges_std = get_edges_std(SG, dir, group, subgroups, ddof=0)
df_edges_std

# Filter by std (std < 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]
df_edges_std_filter """

' df_edges_std = get_edges_std(SG, dir, group, subgroups, ddof=0)\ndf_edges_std\n\n# Filter by std (std < 0.3)\ndf_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]\ndf_edges_std_filter '

time: 1.72 ms (started: 2023-06-12 22:06:32 -05:00)


In [10]:
""" x = df_edges_std["std"]
plt.hist(x, bins=100)
plt.axvline(x=0.3, color="red", lw=1)
l = len(df_edges_std) - len(df_edges_std_filter)
t = len(df_edges_std)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_edges_std["std"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.3, color="red", lw=1)\nl = len(df_edges_std) - len(df_edges_std_filter)\nt = len(df_edges_std)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 14.7 ms (started: 2023-06-12 22:06:32 -05:00)


In [11]:
""" G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])
graph_detail(G)

list_graphs.append(G.copy())
nx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std.gexf".format(dir, group[0])) """

' G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])\ngraph_detail(G)\n\nlist_graphs.append(G.copy())\nnx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std.gexf".format(dir, group[0])) '

time: 12.9 ms (started: 2023-06-12 22:06:32 -05:00)


In [12]:
""" df_edges_filter = nx.to_pandas_edgelist(G)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std.csv".format(dir, group[0]), index=False)
df_edges_filter """

' df_edges_filter = nx.to_pandas_edgelist(G)\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 11.2 ms (started: 2023-06-12 22:06:32 -05:00)


ANOVA

In [13]:
# ANOVA
""" df_nodes_anova = get_nodes_anova(G, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)
df_nodes_anova_filter """

' df_nodes_anova = get_nodes_anova(G, dir, group)\ndf_nodes_anova\n\n# Filter by ANOVA (p > 0.001)\ndf_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]\ndf_nodes_anova_filter\nnodes = list(df_nodes_anova_filter.index)\ndf_nodes_anova_filter '

time: 3.3 ms (started: 2023-06-12 22:06:32 -05:00)


In [14]:
""" x = df_nodes_anova["p-value"]
plt.hist(x, bins=100)
plt.axvline(x=0.001, color="red", lw=1)
l = len(df_nodes_anova) - len(df_nodes_anova_filter)
t = len(df_nodes_anova)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_nodes_anova["p-value"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.001, color="red", lw=1)\nl = len(df_nodes_anova) - len(df_nodes_anova_filter)\nt = len(df_nodes_anova)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 12.9 ms (started: 2023-06-12 22:06:32 -05:00)


In [15]:
""" H = G.subgraph(nodes)
graph_detail(H)
print(len(list(n for n in H.nodes() if H.degree(n) == 0)))

list_graphs.append(H.copy())
nx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std-anova.gexf".format(dir, group[0]))

df_edges_filter = nx.to_pandas_edgelist(H)

df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std-anova.csv".format(dir, group[0]), index=False)
df_edges_filter """

' H = G.subgraph(nodes)\ngraph_detail(H)\nprint(len(list(n for n in H.nodes() if H.degree(n) == 0)))\n\nlist_graphs.append(H.copy())\nnx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std-anova.gexf".format(dir, group[0]))\n\ndf_edges_filter = nx.to_pandas_edgelist(H)\n\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std-anova.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 14.2 ms (started: 2023-06-12 22:06:32 -05:00)


### ANOVA-STD

ANOVA

In [16]:
# ANOVA
""" df_nodes_anova = get_nodes_anova(SG, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)
df_nodes_anova_filter """

' df_nodes_anova = get_nodes_anova(SG, dir, group)\ndf_nodes_anova\n\n# Filter by ANOVA (p > 0.001)\ndf_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]\ndf_nodes_anova_filter\nnodes = list(df_nodes_anova_filter.index)\ndf_nodes_anova_filter '

time: 4.52 ms (started: 2023-06-12 22:06:32 -05:00)


In [17]:
""" x = df_nodes_anova["p-value"]
plt.hist(x, bins=100)
plt.axvline(x=0.001, color="red", lw=1)
l = len(df_nodes_anova) - len(df_nodes_anova_filter)
t = len(df_nodes_anova)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_nodes_anova["p-value"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.001, color="red", lw=1)\nl = len(df_nodes_anova) - len(df_nodes_anova_filter)\nt = len(df_nodes_anova)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 6.33 ms (started: 2023-06-12 22:06:32 -05:00)


In [18]:
# H = SG.subgraph(nodes)

time: 8.84 ms (started: 2023-06-12 22:06:32 -05:00)


In [19]:
""" df_edges_filter = nx.to_pandas_edgelist(H)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova.csv".format(dir, group[0]), index=False)
df_edges_filter """

' df_edges_filter = nx.to_pandas_edgelist(H)\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 10.3 ms (started: 2023-06-12 22:06:33 -05:00)


In [20]:
""" H = nx.from_pandas_edgelist(df_edges_filter, "source", "target")
graph_detail(H)

list_graphs.append(H.copy())
nx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova.gexf".format(dir, group[0])) """

' H = nx.from_pandas_edgelist(df_edges_filter, "source", "target")\ngraph_detail(H)\n\nlist_graphs.append(H.copy())\nnx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova.gexf".format(dir, group[0])) '

time: 2.42 ms (started: 2023-06-12 22:06:33 -05:00)


STD

In [21]:
# STD
""" df_edges_std = get_edges_std(H, dir, group, subgroups, ddof=0)
df_edges_std

# Filter by std (std < 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]
df_edges_std_filter """

' df_edges_std = get_edges_std(H, dir, group, subgroups, ddof=0)\ndf_edges_std\n\n# Filter by std (std < 0.3)\ndf_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]\ndf_edges_std_filter '

time: 13.3 ms (started: 2023-06-12 22:06:33 -05:00)


In [22]:
""" x = df_edges_std["std"]
plt.hist(x, bins=100)
plt.axvline(x=0.3, color="red", lw=1)
l = len(df_edges_std) - len(df_edges_std_filter)
t = len(df_edges_std)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_edges_std["std"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.3, color="red", lw=1)\nl = len(df_edges_std) - len(df_edges_std_filter)\nt = len(df_edges_std)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 14.5 ms (started: 2023-06-12 22:06:33 -05:00)


In [23]:
""" G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])
graph_detail(G)

list_graphs.append(G.copy())
nx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova-std.gexf".format(dir, group[0])) """

' G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])\ngraph_detail(G)\n\nlist_graphs.append(G.copy())\nnx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova-std.gexf".format(dir, group[0])) '

time: 17.4 ms (started: 2023-06-12 22:06:33 -05:00)


In [24]:
""" df_edges_filter = nx.to_pandas_edgelist(G)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova-std.csv".format(dir, group[0]), index=False)
df_edges_filter """

' df_edges_filter = nx.to_pandas_edgelist(G)\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova-std.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 4.68 ms (started: 2023-06-12 22:06:33 -05:00)


### Results

In [25]:
""" for graph in list_graphs:
    graph_detail(graph) """

' for graph in list_graphs:\n    graph_detail(graph) '

time: 11.6 ms (started: 2023-06-12 22:06:33 -05:00)
